# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import torch
from PIL import Image
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
from collections import OrderedDict
import json

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [5]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import torch
from PIL import Image
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
import json



data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
batch_size = 32


# Defining normalized mean and standard deviation 
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]


# Defining data transforms for valid data sets
data_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(norm_mean, norm_std)])
                                         
                                          
# Defining train transforms
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(norm_mean, norm_std)])
                                       

# Defining test transforms
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(norm_mean, norm_std)])
                                          
                                      

# TODO: Load the datasets with ImageFolder
image_datasets = dict()
image_datasets['train'] = datasets.ImageFolder(train_dir, transform=train_transforms)
image_datasets['valid'] = datasets.ImageFolder(valid_dir, transform=data_transforms)
image_datasets['test'] = datasets.ImageFolder(test_dir, transform=test_transforms)


train_dataset = image_datasets['train']
valid_dataset = image_datasets['valid']
test_dataset = image_datasets['test']

# Using the image datasets and trainforms above, define the dataloaders
dataloaders = dict()
dataloaders['train'] = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
dataloaders['valid']  = DataLoader(valid_dataset, batch_size= batch_size, shuffle = True)
dataloaders['test'] = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

trainloader = dataloaders['train']
testloader = dataloaders['test']
validloader = dataloaders['valid']
                                     

# Attach train_data.class_to_idx to class_idx as an attribute to make it
# easier for making inference later on. 

class_idx = train_dataset.class_to_idx   
    

# Print out data to see 
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
class_names = image_datasets['train'].classes

print ("Dataset Size: "+ str(dataset_sizes) + "\n")
n_class = len(class_names)
print ("Number of classes: "+ str(n_class) + "\n")
print ("Classes: "+ str(class_names) + "\n")

print (len(dataloaders))




Dataset Size: {'train': 6552, 'valid': 818, 'test': 819}

Number of classes: 102

Classes: ['1', '10', '100', '101', '102', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']

3


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [3]:

import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [4]:
# DONE: Build and train your network

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import argparse
import torch
from torch.optim import lr_scheduler
from PIL import Image
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
import json


# Load a pre-trained network
model = models.densenet121(pretrained = True)


# Freeze parameters so we don't backpropagate through them and network is not updated
for param in model.parameters():
    param.requires_grad = False

        
input_size = 1024
hidden_layers = '512'
output_size = 102
learning_rate = 0.001

# Defining Network Architecture 
def create_Classifier(model, input_size, hidden_layers, output_size, learning_rate, drop_p=0.5):
    #Freeze feature parameters
    for param in model.parameters():
        param.requires_grad = False
    
    hidden_layers = hidden_layers.split(',')
    hidden_layers = [int(x) for x in hidden_layers]
    hidden_layers.append(output_size)
    
    # Take hidden_layer sizes and creates layer size definitions for each hidden_layer size combination
    layers = nn.ModuleList([nn.Linear(input_size,hidden_layers[0])])
    layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
    layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])   
    
    # Preserves the order in which the keys of net_layers are inserted
    net_layers = OrderedDict()
    
    # Dropout value
    dropout = 0.33
    
    # Creates hidden layers for each size passed by hidden_layers arg
    for x in range(len(layers)):
        layerid = x + 1
        if x == 0:
            net_layers.update({'drop{}'.format(layerid):nn.Dropout(p=dropout)})
            net_layers.update({'fc{}'.format(layerid):layers[x]})
        else:
            net_layers.update({'relu{}'.format(layerid):nn.ReLU()})
            net_layers.update({'drop{}'.format(layerid):nn.Dropout(p=dropout)})
            net_layers.update({'fc{}'.format(layerid):layers[x]})
        
    net_layers.update({'output':nn.LogSoftmax(dim=1)})
    
    #Define classifier
    classifier = nn.Sequential(net_layers)
    
    #Apply new classifier and generate criterion and optimizer
    model.classifier = classifier
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
    
    return model, criterion, optimizer




def train_flower_Classifier(model, trainloader, validloader, criterion, optimizer, epochs, gpu):
    '''
    Trains the selected model based on parameters passed through from command line arguments. 
    This function performs a validation loop every 40 steps, prints progress of the task and 
    returns the trained model. 
    Parameters: 
        model - CNN architecture to be trained
        trainloader - PyTorch training data
        validloader - PyTorch data to be used for validation
        criterion - loss function to be executed
        optimizer - optimizer function to apply the gradients (default - Adam optimizer)
        epochs - number of epochs to tain on 
        gpu - boolean flags to indicate whether GPU or CPU is being used
    Returns:
        model - A trained CNN model 
    '''
    
    steps = 0
    print_every = 40 # A validation loop of 40 steps 
    run_loss = 0 


    # Selects CUDA processing if gpu is True and we have an environment that supports CUDA
    if gpu and torch.cuda.is_available():        
        print("GPU TRAINING")
        model.cuda()        
    elif gpu and torch.cuda.is_available() == False:
        print("GPU processing is selected but no NVIDIA drivers found... Training under CPU.")
    else:
        print("CPU TESTING")
        
    for e in range(epochs): 
        print("Starting to train network...")
        print("Training run {}...".format((e + 1)))
        print("\n")
        
        model.train()
        
# Training forward pass and backpropagation 
        for images, labels in iter(trainloader):        
            steps += 1
            images, labels = Variable(images), Variable(labels)
            if gpu and torch.cuda.is_available():                
                images, labels = images.cuda(), labels.cuda()
                
                # Clear the gradients as the optimizer gets accumulated with each run
                optimizer.zero_grad()
                
                # Forward-pass, backpropagation and then update the weights
                output = model.forward(images)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
                
                run_loss += loss.data.item()
                
                # Runs validation forward pass and loop at specified interval
                if steps % print_every == 0:
                    print("Epoch: {}/{}... ".format(e+1, epochs), "Loss: {:.4f}".format(run_loss/print_every))
                    
                    model.eval()
    
                    accuracy = 0
                    valid_loss = 0
                    
                    for images, labels in iter(validloader):
                        images, labels = Variable(images), Variable(labels)
                    if gpu and torch.cuda.is_available():
                        images, labels = images.cuda(), labels.cuda()
                    # Temporarily turn off gradients (i.e. set requires_grad == False) to preserve memory    
                    with torch.no_grad():
                        out = model.forward(images)
                        valid_loss += criterion(out, labels).data.item()
        
                        # Calculate class probabilities and accuracy
                        # Model's output is log-softmax. Take the exponential to obtain the probabilities
                        ps = torch.exp(out).data
                        # Comparing with true label, class with the highest probability is the predicted class 
                        equality = (labels.data == ps.max(1)[1])
                        # Taking the mean here because accuracy is the number of correct predictions
                        # divided by all predictions
                        accuracy += equality.type_as(torch.FloatTensor()).mean()

        
                    print("Epoch: {}/{}.. ".format(e+1, epochs),
                          "Training Loss: {:.3f}.. ".format(run_loss/print_every),
                          "Valid Loss: {:.3f}.. ".format(valid_loss/len(validloader)),
                          "Valid Accuracy: {:.3f}".format(accuracy/len(validloader)))  
    
                    run_loss = 0
                    model.train()
            
    print("{} EPOCHS COMPLETED. MODEL IS TRAINED.".format(epochs))
    return model 

epochs = 8
gpu = torch.cuda.is_available()


# Create the classifier and train the classifier layers 
    
model, criterion, optimizer = create_Classifier(model,input_size, hidden_layers, output_size, learning_rate, drop_p=0.5)
trained_model = train_flower_Classifier(model, trainloader, validloader, criterion, optimizer, epochs, gpu)

print(trained_model)


/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 101355464.90it/s]


GPU TRAINING
Starting to train network...
Training run 1...


Epoch: 1/8...  Loss: 4.4900
Epoch: 1/8..  Training Loss: 4.490..  Valid Loss: 0.152..  Valid Accuracy: 0.002
Epoch: 1/8...  Loss: 3.9983
Epoch: 1/8..  Training Loss: 3.998..  Valid Loss: 0.135..  Valid Accuracy: 0.011
Epoch: 1/8...  Loss: 3.4996
Epoch: 1/8..  Training Loss: 3.500..  Valid Loss: 0.092..  Valid Accuracy: 0.017
Epoch: 1/8...  Loss: 3.0581
Epoch: 1/8..  Training Loss: 3.058..  Valid Loss: 0.119..  Valid Accuracy: 0.006
Epoch: 1/8...  Loss: 2.8363
Epoch: 1/8..  Training Loss: 2.836..  Valid Loss: 0.053..  Valid Accuracy: 0.036
Starting to train network...
Training run 2...


Epoch: 2/8...  Loss: 2.6625
Epoch: 2/8..  Training Loss: 2.662..  Valid Loss: 0.062..  Valid Accuracy: 0.026
Epoch: 2/8...  Loss: 2.5780
Epoch: 2/8..  Training Loss: 2.578..  Valid Loss: 0.051..  Valid Accuracy: 0.026
Epoch: 2/8...  Loss: 2.5085
Epoch: 2/8..  Training Loss: 2.509..  Valid Loss: 0.066..  Valid Accuracy: 0.026
Epoch: 2/8...  Lo

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [5]:
# DONE: Validation on the test set 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import torch
from PIL import Image
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from torch.optim import lr_scheduler



def test_flower_Classifier(model, testloader, criterion, gpu):
    '''
    Tests the previously trained CNN flower classifier on a test dataset and prints out results.
    Returns nothing. 
    Parameters:
        model - Trained CNN model to test on
        testloader - PyTorch data loader of test data
        criterion - loss function to be executed
        gpu - boolean flag to indicate whether GPU is un use
    Returns:
        None
    '''
        
    # Selecting CUDA if gpu == True and environment supports CUDA
    if gpu and torch.cuda.is_available():
        print("GPU TESTING")
        model.cuda()
    elif gpu and torch.cuda.is_available() == False:
        print("GPU processing is selected but no NVIDIA drivers found... testing under CPU.")
    else:
        print("CPU TESTING")
              
    model.eval()
    
    accuracy = 0 
    run_loss = 0 
    test_loss = 0 
    
    # Forward pass
    for images, labels in iter(testloader):
        images, labels = Variable(images), Variable(labels)
        if gpu and torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        with torch.no_grad():
            output = model.forward(images)
            test_loss += criterion(output, labels).data.item()
            
            # Calculate class probabilities and the accuracy
            # Model's output is log-softmax, take expoential to obtain the probabilities 
            ps = torch.exp(output).data
            # Comparing with true label, class with the highest probability is the predicted class
            equality = (labels.data == ps.max(1)[1])
            # Taking the mean here because accuracy is the number of correct predictions
            # divided by all predictions
            accuracy += equality.type_as(torch.FloatTensor()).mean()
            
    print("Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
           "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
    


# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


#trained_model = train_flower_Classifier(model, trainloader, validloader, criterion, optimizer, epochs, gpu)
test_flower_Classifier(model, testloader, criterion, gpu)

GPU TESTING
Test Loss: 0.965..  Test Accuracy: 0.742


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [6]:
# DONE: Save the checkpoint 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import torch
from PIL import Image
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from torch.optim import lr_scheduler

num_epochs = 10
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model.class_to_idx = dataloaders['train'].dataset.class_to_idx
model.epochs = num_epochs
checkpoint = {'input_size': [3, 224, 224],
                 'batch_size': dataloaders['train'].batch_size,
                  'output_size': 102,
                  'state_dict': model.state_dict(),
                  'optimizer_dict':optimizer.state_dict(),
                  'class_to_idx': model.class_to_idx,
                  'epoch': model.epochs}
torch.save(checkpoint, 'checkpoint.pth')    



## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [7]:
# A function that loads a checkpoint and rebuilds the model

ckpt = torch.load('checkpoint.pth')
ckpt.keys()

def load_checkpoint(file_dir):
    
    checkpoint = torch.load(file_dir)

    # Load model parameters
    
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']

        
    # Set to False so that network does not update in every iteration
    for x in model.parameters(): 
        x.requires_grad = False
        
    return model, checkpoint['class_to_idx']


    
# get index to class mapping
loaded_model, class_to_idx = load_checkpoint('checkpoint.pth')
idx_to_class = { v : k for k,v in class_to_idx.items()}

print(loaded_model)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [8]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import torch

from PIL import Image
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict



# Processing the image
def process_image(image):
    ''' 
    This function scales, crops and normalizes a PIL image. It then resizes 
    and transposes the image into a new image for PyTorch model. 
    Returns - a Numpy array
    '''
    
    norm_mean = [0.485, 0.456, 0.406]
    norm_std = [0.229, 0.224, 0.225]
        
    # Process a PIL image for use in a PyTorch model
    img = Image.open(image)
       
    # Creates a 256 x 256 thumbnail of the JPEG image
    img.thumbnail((256,256), Image.ANTIALIAS)
    
    # Cropping the image 
    img = img.crop((16, 16, 240, 240))
    
    img = np.array(img)
    
    # Normalize and resize array
    mean = np.array(norm_mean)
    std = np.array(norm_std)
    np_image = img/255
    np_image_array = (np_image - mean)/std
    
    # PyTorch tensors expect the color channel to be the first dimension but
    # it's the third dimension in PIL image and Numpy array
    np_image_array = np_image.transpose((2, 0, 1))
    
    # Convert Numpy array to a Tensor array
#    np_image_array = torch.tensor(np_image_array)
    
    return np_image_array
   


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [9]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [10]:
import numpy as np
import torch
from torchvision import datasets, transforms, models
from PIL import Image


data_dir = 'flowers'
test_dir = data_dir + '/test'
image_path = test_dir + '/16/image_06673.jpg'

model = models.densenet121(pretrained=True)
def predict(image_path, model, topk=5):
    '''
    Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # Predicting the class (or classes) from an image file
    
    
    image = torch.FloatTensor(process_image(image_path))
    # Convert 2D image to a 1D vector
    image = np.expand_dims(image,0)
    
    image = torch.from_numpy(image)
    
    model.eval()
    output = model.forward(Variable(image))
    probabilities = torch.exp(output).data.numpy()[0]
    

    top_idx = np.argsort(probabilities)[-topk:][::-1] 
    top_class = [idx_to_class[x] for x in top_idx]
    top_probability = probabilities[top_idx]

 
    return top_probability, top_class



/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [11]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import torch

from PIL import Image
from PIL import ImageFile
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
 

data_dir = 'flowers'
test_dir = data_dir + '/test'
image_path = test_dir + '/18/image_04322.jpg'
print("The image path is = {}".format(image_path))

model = models.densenet121(pretrained=True)


# TODO: Display an image along with the top 5 classes 
def view_classify (img, probabilities, classes, mapper): 
    
    
    img_filename = img.split (' / ') [- 2] 
    img = Image.open (img)
    fig, (ax1, ax2) = plt.subplots (figsize = (6,10), numcols = 1, numrows = 2) 
    flower_name = mapper [img_filename] 
    
    ax1.set_title (flower_name) 
    ax1.imshow (img) 
    ax1.axis ( 'off') 
    
    y_pos = np.arange (len (probabilities)) 
    ax2.barh (y_pos, probabilities) 
    ax2.set_yticks (y_pos) 
    ax2.set_yticklabels ([mapper [x] for x in classes]) 
    ax2.invert_yaxis ()
     
img = image_path 
probabilities, classes = predict(img, model) 

view_classify(img, probabilities, classes, cat_to_name)



The image path is = flowers/test/18/image_04322.jpg


/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


KeyError: 985

In [12]:
img = 'flowers/test/18/image_04322.jpg'
p, c = predict(img, loaded_model)
view_classify(img, p, c, cat_to_name)

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.FloatTensor for argument #2 'weight'